# 02. 로지스틱 회귀 (Logistic Regression)

## 학습 목표
- 로지스틱 회귀의 원리 이해 (이진 분류)
- 시그모이드 함수와 확률 해석
- 분류 평가 지표 (정확도, 정밀도, 재현율, F1)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, confusion_matrix, classification_report,
                             roc_curve, roc_auc_score)
from sklearn.datasets import make_classification, load_iris
import seaborn as sns

plt.rcParams['font.family'] = 'DejaVu Sans'

## 1. 시그모이드 함수 이해

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# 시그모이드 함수 시각화
z = np.linspace(-10, 10, 100)
plt.figure(figsize=(10, 5))
plt.plot(z, sigmoid(z), linewidth=2)
plt.axhline(y=0.5, color='r', linestyle='--', alpha=0.5)
plt.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
plt.xlabel('z')
plt.ylabel('σ(z)')
plt.title('Sigmoid Function: σ(z) = 1 / (1 + e^(-z))')
plt.grid(True, alpha=0.3)
plt.show()

## 2. 이진 분류 데이터 생성

In [ ]:
# 인공 데이터 생성
X, y = make_classification(
    n_samples=500, 
    n_features=2,
    n_redundant=0,
    n_informative=2,
    n_clusters_per_class=1,
    random_state=42
)

print(f"X shape: {X.shape}")
print(f"y classes: {np.unique(y)}")
print(f"Class distribution: {np.bincount(y)}")

In [ ]:
# 데이터 시각화
plt.figure(figsize=(10, 6))
plt.scatter(X[y==0, 0], X[y==0, 1], c='blue', label='Class 0', alpha=0.6, edgecolors='black')
plt.scatter(X[y==1, 0], X[y==1, 1], c='red', label='Class 1', alpha=0.6, edgecolors='black')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Binary Classification Data')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 3. 모델 학습

In [ ]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 로지스틱 회귀 모델
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

print(f"Coefficients: {model.coef_}")
print(f"Intercept: {model.intercept_}")

## 4. 예측 및 평가

In [ ]:
# 예측
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]  # 클래스 1의 확률

# 평가 지표
print("=== Classification Metrics ===")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_test, y_prob):.4f}")

In [ ]:
# Classification Report
print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred))

In [ ]:
# 혼동 행렬 시각화
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Predicted 0', 'Predicted 1'],
            yticklabels=['Actual 0', 'Actual 1'])
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

print(f"\nTrue Negatives: {cm[0,0]}")
print(f"False Positives: {cm[0,1]}")
print(f"False Negatives: {cm[1,0]}")
print(f"True Positives: {cm[1,1]}")

## 5. ROC 곡선

In [ ]:
# ROC 곡선
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
auc = roc_auc_score(y_test, y_prob)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, linewidth=2, label=f'ROC Curve (AUC = {auc:.3f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 6. 결정 경계 시각화

In [ ]:
def plot_decision_boundary(model, X, y, title='Decision Boundary'):
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                         np.linspace(y_min, y_max, 200))
    
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    plt.figure(figsize=(10, 6))
    plt.contourf(xx, yy, Z, alpha=0.3, cmap='RdYlBu')
    plt.scatter(X[y==0, 0], X[y==0, 1], c='blue', label='Class 0', edgecolors='black')
    plt.scatter(X[y==1, 0], X[y==1, 1], c='red', label='Class 1', edgecolors='black')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.title(title)
    plt.legend()
    plt.show()

plot_decision_boundary(model, X_test, y_test, 'Logistic Regression Decision Boundary')

## 7. 다중 클래스 분류 (Iris Dataset)

In [ ]:
# Iris 데이터 로드
iris = load_iris()
X_iris = iris.data
y_iris = iris.target

print(f"Features: {iris.feature_names}")
print(f"Classes: {iris.target_names}")
print(f"Shape: {X_iris.shape}")

In [ ]:
# 다중 클래스 로지스틱 회귀
X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(
    X_iris, y_iris, test_size=0.3, random_state=42
)

model_multi = LogisticRegression(multi_class='multinomial', max_iter=200, random_state=42)
model_multi.fit(X_train_i, y_train_i)

y_pred_i = model_multi.predict(X_test_i)
print(f"Accuracy: {accuracy_score(y_test_i, y_pred_i):.4f}")
print(f"\nClassification Report:")
print(classification_report(y_test_i, y_pred_i, target_names=iris.target_names))

In [ ]:
# 다중 클래스 혼동 행렬
cm_multi = confusion_matrix(y_test_i, y_pred_i)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_multi, annot=True, fmt='d', cmap='Blues',
            xticklabels=iris.target_names,
            yticklabels=iris.target_names)
plt.title('Confusion Matrix - Iris Dataset')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

## 정리

### 핵심 개념
- **로지스틱 회귀**: 시그모이드 함수로 확률 출력 (0~1)
- **결정 경계**: P(y=1) = 0.5인 지점
- **평가 지표**:
  - Accuracy: 전체 정확도
  - Precision: 양성 예측 중 실제 양성 비율
  - Recall: 실제 양성 중 예측 성공 비율
  - F1 Score: Precision과 Recall의 조화 평균

### 다음 단계
- 규제 (L1/L2 Regularization)
- 임계값 조정
- 불균형 데이터 처리